# Machine Learning: Random Forest

## 0 Imports

In [1]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import confusion_matrix, recall_score, matthews_corrcoef
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [6]:
data_change = pd.read_csv("../../dat/dips/DIPS_Data_cleaned_change.csv", sep =",", low_memory = False)
data_change_excl = pd.read_csv("../../dat/dips/DIPS_Data_cleaned_change_excl.csv", sep =",", low_memory = False)

## 1 Random Forest Model

In [7]:
# Predictors
X = data_change_excl
X = X.drop(["hpi"], axis = 1)

# Target
y = data_change_excl["hpi"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

print("Random Forest \n",
      "X_train shape: ", X_train.shape, round(X_train.shape[0]/len(X), 2), "\n",
      "X_test shape: ", X_test.shape, round(X_test.shape[0]/len(X), 2),  "\n",
      "y_train shape: ", y_train.shape, round(y_train.shape[0]/len(y), 2), "\n",
      "y_test shape: ", y_test.shape, round(y_test.shape[0]/len(y), 2), "\n")

Random Forest 
 X_train shape:  (785, 23) 0.8 
 X_test shape:  (197, 23) 0.2 
 y_train shape:  (785,) 0.8 
 y_test shape:  (197,) 0.2 



In [8]:
X_train

,Unnamed: 0,whi_ges,soz_ges,gke_ges,lzh_ges,ile,woc_gesp,woc_gese,asi_ges,bsq_ges,...,bmi_kat,ses_kom,whi_change,soz_change,gke_change,lzh_change,asi_change,bsq_change,das_change,scl_change
786,1198,-0.467185,-1.170941,1.722078,-1.054628,0.114940,0.430163,0.069395,1.838983,2.260436,...,0,0,0.273386,0.675072,-1.050943,0.151192,2.842393,1.004148,2.645099,0.316829
470,725,2.095807,-0.520217,0.075914,-0.079306,0.114940,0.607093,0.650418,0.218995,0.134855,...,0,-1,-3.068499,0.331764,-0.766089,-0.726937,-1.004358,-0.852021,0.649488,-0.581447
878,1367,1.070610,0.595309,-0.394418,0.245802,0.731724,-0.366021,0.816425,0.893990,1.680732,...,-1,0,-0.840576,-0.011545,-1.050943,-0.200059,0.094714,1.004148,0.421418,-0.132309
707,1077,0.558012,0.223467,1.016579,0.570909,0.731724,-1.516066,0.235402,1.028989,0.714559,...,0,0,-0.283595,-0.125981,0.943035,0.151192,-0.180054,0.222603,0.250366,-0.232117
767,1171,0.045414,0.874190,1.016579,1.383678,-0.501843,1.137883,0.401408,-0.860997,-1.121170,...,-1,0,-0.283595,-2.071397,0.658181,-0.200059,0.232098,0.124910,-0.319809,0.416637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
679,1033,-0.979783,-0.613178,1.251745,0.570909,0.114940,-1.604531,0.816425,-1.265994,-0.734701,...,0,0,0.273386,1.590562,0.943035,0.853696,-0.042670,0.417989,-0.490861,-0.282022
227,345,0.045414,0.595309,0.075914,-0.892074,0.731724,-0.542951,-0.013608,-0.321001,-0.154997,...,0,0,1.387347,-0.469290,0.943035,1.029322,-0.592206,-0.363556,-0.205774,-0.282022
415,639,-0.467185,0.316427,0.546247,0.570909,-0.913033,-0.366021,-0.179615,0.623992,1.390880,...,0,0,-0.283595,0.217327,-0.481235,-0.024433,0.369482,-0.070476,-0.319809,0.217020
961,1506,-0.979783,-1.542783,-0.394418,-0.729520,-0.090654,1.137883,0.899428,-0.590999,-1.121170,...,0,0,1.387347,1.933871,-0.196381,-0.024433,0.232098,0.515682,-0.832966,0.366733


In [9]:
X_test

,Unnamed: 0,whi_ges,soz_ges,gke_ges,lzh_ges,ile,woc_gesp,woc_gese,asi_ges,bsq_ges,...,bmi_kat,ses_kom,whi_change,soz_change,gke_change,lzh_change,asi_change,bsq_change,das_change,scl_change
930,1455,1.070610,-1.170941,-0.159252,-0.079306,1.142913,1.226347,1.231442,0.488993,-0.058380,...,0,0,-0.840576,1.018381,0.373327,-0.200059,-0.042670,1.297227,-0.718931,-0.082405
612,931,-0.979783,1.060111,0.781413,1.383678,-0.501843,-1.250671,-1.839682,-1.535992,-1.314404,...,0,0,0.830367,-1.041471,-0.196381,-0.375685,0.094714,0.417989,0.364401,0.416637
217,332,0.045414,0.688269,0.781413,-0.566967,0.526130,0.607093,-0.594632,-0.456000,-1.121170,...,0,-1,-0.283595,-0.011545,0.658181,0.678070,-0.729590,0.027217,0.250366,0.316829
656,997,1.583209,0.409388,0.311080,0.083248,-0.090654,-1.516066,-0.179615,0.083996,0.907793,...,0,0,-1.954538,0.217327,-0.196381,-0.200059,-0.454822,-0.852021,0.763523,0.017404
761,1161,-0.467185,1.060111,1.251745,1.221124,0.731724,2.022532,0.650418,-1.535992,-1.024552,...,0,-1,0.273386,-0.240417,0.658181,-1.253815,0.094714,-0.070476,-0.718931,0.067308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,275,1.583209,0.223467,-0.629585,0.733463,-0.296249,-0.277556,-1.092652,-0.590999,0.231472,...,0,0,-0.283595,0.560636,1.227889,-0.551311,0.644250,0.027217,-0.091739,0.266925
548,834,-0.467185,-0.148375,-1.335083,-0.241859,-0.296249,-0.631416,-0.096612,-0.051003,-1.024552,...,0,0,-0.283595,-0.240417,-0.196381,0.502444,0.644250,1.297227,1.333697,0.067308
617,938,0.045414,0.037546,0.075914,0.408356,-0.296249,-0.277556,-0.511629,0.218995,-0.058380,...,0,0,-0.840576,-1.041471,-1.050943,-0.726937,0.369482,-0.754328,-0.091739,-0.481638
762,1162,1.583209,1.060111,-0.394418,-0.404413,-0.501843,-1.250671,-0.511629,0.218995,-0.348231,...,0,0,0.830367,-0.240417,-0.481235,0.502444,-1.279126,-0.461249,0.421418,-0.607533


In [4]:
def sklearn_random_forest(x_test_rf, x_train_rf, y_train_rf, y_test_rf):

    # 1: GRID SEARCH
    rfc_model = RandomForestClassifier(random_state = 42)

    # param_grid = {
    #     'n_estimators': [100, 200, 300],
    #     'max_depth': [None, 10, 20, 30],
    #     'min_samples_split': [2, 5, 10],
    #     'min_samples_leaf': [1, 2, 4]
    # }

    param_grid = {
        'bootstrap': [True],
        'max_depth': [None, 10, 30, 50],
        'max_features': [2, 5, 10],
        'min_samples_leaf': [1, 3, 5],
        'min_samples_split': [2, 5, 10, 12],
        'n_estimators': [100, 200, 300, 500],
        'criterion': ['gini', 'entropy']
    }

    grid_search = GridSearchCV(
        estimator = rfc_model,
        param_grid = param_grid,
        cv = 10
    )
    grid_search.fit(x_train_rf, y_train_rf)
    best_model = grid_search.best_estimator_
    print("Best parameters: ", grid_search.best_params_)
    print("Best score: ", grid_search.best_score_)

    # 2: FITTING THE MODEL
    model = RandomForestClassifier(
        bootstrap = best_model.bootstrap,
        max_depth = best_model.max_depth,
        max_features = best_model.max_features,
        min_samples_leaf = best_model.min_samples_leaf,
        min_samples_split = best_model.min_samples_split,
        n_estimators = best_model.n_estimators,
        criterion = best_model.criterion,
        random_state = 42
    )
    model.fit(x_train_rf, y_train_rf)

    # 3: ESTIMATING WEIGHTS
    weights_randf = model.feature_importances_
    features_randf = model.feature_names_in_

    # 4: COMPUTE TEST SET PREDICTIONS
    y_pred_randf = model.predict(x_test_rf)
    y_pred_proba_randf = model.predict_proba(x_test_rf)

    # 5: COMPUTE METRICS
    accuracy_rf = model.score(x_test_rf, y_test_rf)
    macro_f1_rf = recall_score(y_test_rf, y_pred_randf, average = "macro")
    micro_f1_rf = recall_score(y_test_rf, y_pred_randf, average = "micro")
    mcc_rf = matthews_corrcoef(y_test_rf, y_pred_randf)

    cm_rf = confusion_matrix(y_test_rf, y_pred_randf)
    precision_rf = cm_rf[1][1] / (cm_rf[1][1] + cm_rf[0][1])
    recall_rf = cm_rf[1][1] / (cm_rf[1][1] + cm_rf[1][0])

    # store metrics in a dictionary
    metrics_randf = {
        "accuracy": round(accuracy_rf, 4),
        "macro_f1": round(macro_f1_rf, 4),
        "micro_f1": round(micro_f1_rf, 4),
        "mcc": round(mcc_rf, 4),
        "precision": round(precision_rf, 4),
        "recall": round(recall_rf, 4),
        "confusion_matrix": cm_rf
    }

    return weights_randf, y_pred_randf, y_pred_proba_randf, features_randf, metrics_randf

In [5]:
start = time.time()

weights_rf, y_pred_rf, y_pred_proba_rf, features_rf, metrics_rf = sklearn_random_forest(X_test, X_train, y_train, y_test)

end = time.time()
print("Execution time: ", {end - start}, "seconds")
time_rf = end - start
time_rf_df = pd.DataFrame({"time": [time_rf]})
time_rf_df.to_csv("../exp/times_ML/time_rf.csv", sep = ",", index = False)

# save weights_ML and y_pred_ML
weights_rf = pd.DataFrame([weights_rf], columns = features_rf)
weights_rf.to_csv("weights_ML/weights_rf.csv", sep = ",", index = False)

y_pred_rf = pd.DataFrame(y_pred_rf, columns = ["y_pred"])
y_pred_rf.to_csv("y_pred_ML/y_pred_rf.csv", sep = ",", index = False)

Best parameters:  {'bootstrap': True, 'criterion': 'gini', 'max_depth': 10, 'max_features': 5, 'min_samples_leaf': 1, 'min_samples_split': 12, 'n_estimators': 100}
Best score:  0.7364254792826221
Execution time:  {7655.865532875061} seconds


In [6]:
metrics_rf

{'accuracy': 0.7276,
 'macro_f1': 0.5676,
 'micro_f1': 0.7276,
 'mcc': np.float64(0.1974),
 'precision': np.float64(0.5385),
 'recall': np.float64(0.2029),
 'confusion_matrix': array([[165,  12],
        [ 55,  14]])}